In [14]:
pipenv

In [ ]:
!pip freeze | grep scikit-learn

In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [8]:
# Q1: What's the mean predicted duration for this dataset?
y_pred.mean()

16.191691679979066

In [11]:
# Q2. Preparing the output. What's the size of the output file?
year = 2021
month = 2
output_file = 'predictions.parquet'
df['predicted_duration'] = y_pred
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
df_result = df[['predicted_duration','ride_id']]
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
# 19.7 MB

In [12]:
# Q3. Creating the scoring script
# jupyter nbconvert --to script starter.ipynb

In [15]:
# Q4. Virtual environment - What's the first hash for the Scikit-Learn dependency?
# sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

In [16]:
# Q5. Parametrize the script
# Let's now make the script configurable via CLI. We'll create two parameters: year and month. Run the script for March 2021. What's the mean predicted duration?
# mean predicted duration: 16.298821614015107

In [17]:
# Q6. Docker container
# Build container using docker build -t mlops-learn .
# run it using  docker run -it --rm mlops-learn
# mean predicted duration: 9.967573179784523